In [47]:
import pandas as pd
import geopandas as gpd
import h3
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

In [73]:
cowpea = pd.read_csv('cowpea_gridcell_summary.csv')
cowpea_gdf = gpd.GeoDataFrame(cowpea, geometry=gpd.points_from_xy(cowpea.long, cowpea.lat))
cowpea_gdf.set_crs(epsg=4326, inplace=True)
cowpea_gdf

,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,geometry
0,114115,-2.75,10.75,1885,1842,1411,1943,1663,-2.28,-25.15,3.08,-11.78,POINT (-2.75000 10.75000)
1,114115,-2.75,10.75,1767,1979,1775,1701,1683,12.00,0.45,-3.74,-4.75,POINT (-2.75000 10.75000)
2,114115,-2.75,10.75,1742,1823,1870,1767,1540,4.65,7.35,1.44,-11.60,POINT (-2.75000 10.75000)
3,114115,-2.75,10.75,1766,1870,1881,1381,1392,5.89,6.51,-21.80,-21.18,POINT (-2.75000 10.75000)
4,114115,-2.75,10.75,1834,1997,1902,1588,1786,8.89,3.71,-13.41,-2.62,POINT (-2.75000 10.75000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,120601,0.25,6.25,1557,745,1626,1580,1141,-52.15,4.43,1.48,-26.72,POINT (0.25000 6.25000)
1016,120601,0.25,6.25,1625,1690,1542,1618,1341,4.00,-5.11,-0.43,-17.48,POINT (0.25000 6.25000)
1017,120601,0.25,6.25,1486,1668,1641,1307,1032,12.25,10.43,-12.05,-30.55,POINT (0.25000 6.25000)
1018,120601,0.25,6.25,1621,1760,1567,1513,1150,8.57,-3.33,-6.66,-29.06,POINT (0.25000 6.25000)


In [39]:
# in cowpea_gdf combine all rows that share exp values and take the mean of the other columns
cowpea_mean = cowpea.groupby('exp').mean()
cowpea_min = cowpea.groupby('exp').min()
cowpea_max = cowpea.groupby('exp').max()

In [42]:
cowpea_mean_gdf = gpd.GeoDataFrame(cowpea_mean, geometry=gpd.points_from_xy(cowpea_mean.long, cowpea_mean.lat))
cowpea_mean_gdf.set_crs(epsg=4326, inplace=True)
cowpea_mean_gdf.explore()

In [46]:
# cowpea_mean_gdf unique geometry values
print(len(cowpea_mean_gdf.geometry.unique()))
print(len(cowpea_mean_gdf.geometry))

51
51


In [62]:
tester = cowpea_mean_gdf.copy()

In [81]:

def points_to_grid (points_gdf, edge_size):
    square_gdf = points_gdf.copy()
    for index, row in square_gdf.iterrows():
        point = row['geometry']
        
        # Define the coordinates for the corners of the square
        x, y = point.x, point.y
        square_half_size = edge_size*0.5
        
        # Create the coordinates for the square's corners
        square_coords = [(x - square_half_size, y - square_half_size),
                        (x + square_half_size, y - square_half_size),
                        (x + square_half_size, y + square_half_size),
                        (x - square_half_size, y + square_half_size)]
        
        polygon = Polygon(square_coords)
        square_gdf.loc[index, 'geometry'] = polygon
    return square_gdf



In [84]:
# write tester to geojson
tester.to_file("cowpea_gridcell_summary_poly.geojson", driver='GeoJSON')


In [64]:

polygons = []
attributes = []

# Iterate through each point in the original GeoDataFrame
for index, row in tester.iterrows():
    point = row['geometry']  # Get the point geometry
    
    # Define the coordinates for the corners of the square
    x, y = point.x, point.y
    square_half_size = 0.25  # Half of 0.25 degrees
    
    # Create the coordinates for the square's corners
    square_coords = [(x - square_half_size, y - square_half_size),
                     (x + square_half_size, y - square_half_size),
                     (x + square_half_size, y + square_half_size),
                     (x - square_half_size, y + square_half_size)]
    
    # Create a Polygon geometry from the coordinates
    polygon = Polygon(square_coords)
    polygons.append(polygon)

# Create a new GeoDataFrame with the polygons
squares_gdf = gpd.GeoDataFrame(geometry=polygons, crs=tester.crs)
# replace tester geometry with squares_gdf geometry
tester['geometry'] = squares_gdf['geometry']

In [65]:
squares_gdf

,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,geometry
exp,,,,,,,,,,,,
114115,-2.75,10.75,1770.00,1858.80,1756.05,1614.65,1386.50,5.1250,-0.6590,-8.7290,-21.6865,None
114116,-2.25,10.75,1762.95,1845.15,1771.40,1638.95,1419.50,4.7620,0.5400,-6.9335,-19.3730,None
114117,-1.75,10.75,1719.05,1767.20,1689.50,1520.05,1285.55,2.9710,-1.5430,-11.5285,-25.1335,None
114118,-1.25,10.75,1686.70,1685.60,1558.60,1422.35,1156.70,0.0770,-7.2415,-15.4780,-31.2530,None
114119,-0.75,10.75,1693.70,1709.60,1619.15,1450.35,1189.20,1.0900,-4.0335,-14.2590,-29.6895,None
114120,-0.25,10.75,1696.80,1708.70,1623.95,1464.20,1222.30,0.8500,-4.0005,-13.6920,-27.8520,None
114835,-2.75,10.25,1751.15,1822.45,1745.65,1610.60,1397.60,4.3245,-0.0975,-7.9195,-20.0025,None
114836,-2.25,10.25,1738.20,1825.70,1785.20,1646.45,1446.90,5.1835,2.7500,-5.1825,-16.6935,None
114837,-1.75,10.25,1711.45,1773.90,1704.90,1551.30,1328.65,3.8235,-0.3015,-9.2885,-22.3160,None


In [8]:
print("Max: ", cowpea['lat'].max())
print("Min: ", cowpea['lat'].min())
print("Max: ", cowpea['long'].max())
print("Min: ", cowpea['long'].min())

Max:  10.75
Min:  6.25
Max:  0.25
Min:  -2.75


In [ ]:
# for reference
lat, lng = 37.769377, -122.388903
resolution = 9
h3.latlng_to_cell(lat, lng, resolution)